In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import carbonplan_styles

carbonplan_styles.mpl.set_theme(font_scale=1.25)
plt.rcParams["font.family"] = "sans"

In [ ]:
def load_data(fn):
    data = pd.read_csv(fn)
    return data.set_index(["pltID", "YEAR"])


def merge_species_plot(species_df, plot_df, species_code=631):
    species = species_df[species_df["SPCD"] == species_code]
    return species.join(plot_df, rsuffix="_plot", lsuffix="_species")

In [ ]:
plot_biomass = load_data(
    "gs://carbonplan-buffer-analysis/rfia/plot_biomass.csv"
)
species_biomass = load_data(
    "gs://carbonplan-buffer-analysis/rfia/plot_species_biomass.csv"
)
plot_tpa = load_data("gs://carbonplan-buffer-analysis/rfia/plot_tpa.csv")
species_tpa = load_data(
    "gs://carbonplan-buffer-analysis/rfia/plot_species_tpa.csv"
)

In [ ]:
tanoak_biomass = merge_species_plot(species_biomass, plot_biomass)
tanoak_tpa = merge_species_plot(species_tpa, plot_tpa)

perc_baa = tanoak_tpa["BAA_species"] / tanoak_tpa["BAA_plot"]
perc_biomass = (
    tanoak_biomass["BIO_ACRE_species"] / tanoak_biomass["BIO_ACRE_plot"]
)

tp = (
    pd.concat(
        [perc_baa.rename("perc_baa"), perc_biomass.rename("perc_biomass")],
        axis=1,
    )
    * 100
)

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3.5))
ax.scatter(tp["perc_baa"], tp["perc_biomass"], color=".3", s=70, alpha=0.6)
ax.plot((0, 100), (0, 100), carbonplan_styles.colors.light["secondary"])
plt.xlim(0, 105)
plt.ylim(0, 105)

plt.xlabel("Tanoak Basal Area\n(% Plot Total)")
plt.ylabel("Tanoak Biomass\n(% Plot Total)")

ax.set_xticks(range(0, 101, 50))
ax.set_yticks(range(0, 101, 50))

carbon = "#1B1E23"
[ax.spines[spine].set_color(carbon) for spine in ax.spines]
ax.tick_params(axis="x", colors=carbon)
ax.tick_params(axis="y", colors=carbon)
plt.savefig("../img/tanoak_fia_methods.pdf", dpi=300, bbox_inches="tight")